In [ ]:
from dataloader import CreateDataset

In [ ]:
from bert3head.model import Linear3HEAD
import torch

model = Linear3HEAD("vinai/phobert-base-v2")
model.load_state_dict(torch.load('model/linear/3head-boosting2.pt'))
model = model.cuda()

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('label_gemini_vi_v2.csv')
dataloader = CreateDataset(df['text'], df['sentiment'], df['classification'],"vinai/phobert-base-v2",64, shuffle=False).label()
sentences = []
sentiments = []
classifications = []

model.eval()
with torch.no_grad():
    for sentence, input_ids, attention_mask in dataloader:
        sentences.extend(sentence)
        input_ids = input_ids.cuda()
        attention_mask = attention_mask.cuda()
        
        sen, clas = model(input_ids, attention_mask)
        sen = sen.detach().cpu().numpy()
        clas = clas.detach().cpu().numpy()
        
        sen = sen.argmax(axis=1).flatten()
        clas = clas.argmax(axis=1).flatten()
        
        sentiments.extend(sen)
        classifications.extend(clas)
        
sentences = np.array(sentences)
sentiments = np.array(sentiments)
classifications = np.array(classifications)
new_df = pd.DataFrame({'text':sentences, 'sentiment':sentiments, 'classification':classifications})
new_df.to_csv('label_gemini_vi_v2_linear3head.csv', index=False)